# Steinschlagrisiko cwm1

## Aufgabenstellung
Die Kantonsstrasse unterhalb Schiers (GR) ist vom Steinschlag betroffen. Steine lösen sich von zwei unterschiedlichen Stellen an der Felswand ab (Ablösungszone 1 und Ablösungszone 2). Der betroffene Strassenabschnitt ist mit Steinfangnetzen gesichert, die jedoch in die Jahre gekommen sind und die angestrebte Sicherheit nicht mehr gewährleisten können. Die Planung für Ersatznetze hat bereits begonnen, kann aber frühstens in einem Jahr umgesetzt werden.

In den letzten Monaten haben sich mehrere Steinschlagereignisse ereignet. Kommt es im Lauf des nächsten Jahres zu weiteren vergleichbaren Ereignissen, könnten die alten Sicherheitsnetze versagen und die Verkehrsteilnehmer einem grossen Sicherheitsrisiko ausgesetzt sein. Die Bevölkerung ist verunsichert und der Kantonsingenieur muss schnell entscheiden, ob das Risiko für die Verkehrsteilnehmer zu gross ist und die Kantonsstrasse vorübergehend gesperrt werden muss. Der Kantonsingenieur hat sie beauftragt, anhand von vorhanden Daten die Wahrscheinlichkeit eines Todesfalls zu berechnen und eine Empfehlung bezüglich der Schliessung bzw Offenhaltung der Strasse auszusprechen.

Damit die Strasse offen bleiben kann, muss gezeigt werden, dass die jährliche Wahrscheinlichkeit von Todesfällen infolge Steinschlags kleiner als 10-4  ist. Für die Berechnungen soll ein gut strukturierter und dokumentierter Code in Python oder R entwickelt werden.


## Grundlagen
Für die Planung der neuen Sicherheitsnetze, hat ein beauftragter Geologe, über drei Monate Daten zu den Steinschlagereignisse aufgenommen. Dabei wurde Steingeschwindigkeit, Steinmasse und Zeitpunkt und der Zeitpunkt des Ereignis registriert. Diese Daten können von Ihnen zur Modellbildung verwendet werden und stehen unter folgenden Links zur Verfügung:

Ereignis aus Ablösungszone 1: https://www.dropbox.com/s/i58gdv6pzi03rhr/out_1.csv?dl=0

Ereignis aus Ablösungszone 2: https://www.dropbox.com/s/3nk9pv7nzz8f0qb/out_2.csv?dl=0
NB: Die Geschwindigkeit ist durch einen Radar aufgenommen und sehr präzise. Die Masse ist eine Experten-Schätzung des Geologen.
Ein beauftragtes Ingenieurbüro hat geschätzt, dass die Sicherheitsnetze bis zu einer Aufprallenergie von 1000 kJ sicher sind. Falls bereits ein Stein mit über 2000kg in den Sicherheitsnetzen liegt, beträgt die Aufprallenergie, die von den Sicherheitsnetzen aufgenommen werden kann, nur noch 500 kJ. Steine in den Sicherheitsnetze werden vom Unterhaltsteam entfernt (die Reaktionszeit beträgt 24 Stunden).
Das tägliche Verkehrsaufkommen beträgt 1200 Autos. Stau kommt auf der Strecke nicht vor. Die Tempolimite beträgt 60 km/h. 

## Abgabe
Form der Abgabe
Quellcode der Berechnungen in einem Notebook (R oder Jupyter).

Kurzer Bericht (max 8 Seiten) der die Berechnungen klar und nachvollziebar beschreibt und die Empfehlung an den Kantonsingenieur begründet.

### Abgabetermin
Montag, 10. Januar 2022

## 1. Bibliotheken Importieren

In [1]:
import numpy as np
import random as rnd
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from msticpy.nbtools.timeline import display_timeline

ModuleNotFoundError: No module named 'scipy'

## 2. Daten einlesen

In [ ]:
df1 = pd.read_csv("out_1.csv") # Daten einlesen
df2 = pd.read_csv("out_2.csv")

## 3. Daten Bereinigung und Bearbeiten

In [ ]:
df1 = df1.loc[:, ["Datum","Uhrzeit","Masse [kg]","Geschwindigkeit [m/s]"]] # Datentabelle auswaehlen
df1 = df1.dropna() # entfernt alle Zeile die mit NA Werte
df1['Ablösungszone 1'] = 1 # Spalte Ablösungszone 1 hinzugefuegt

In [ ]:
df2 = df2.rename(columns={"Date":"Datum","m [kg]":"Masse [kg]","v [m/s]":"Geschwindigkeit [m/s]"}) # Spaltenname aendern
df2 = df2.loc[:, ["Datum","Uhrzeit","Masse [kg]","Geschwindigkeit [m/s]"]] # Datentabelle auswaehlen
df2 = df2.dropna() # entfernt alle Zeile die mit NA Werte
df2['Ablösungszone 2'] = 2 #Spalte Ablösungszone 2 hinzugefuegt

In [ ]:
df1.describe()

## 4. Tabellen zusammenfuegen

In [ ]:
frames = [df1, df2]
df3 = pd.concat(frames)
df3

## 5. Kinetische Energie

In [ ]:
liste_masse = [(x) for x in df3['Masse [kg]']]
liste_geschwindigkeit = [(x) for x in df3["Geschwindigkeit [m/s]"]]
liste_energie = [] 

for i in range(len(liste_masse)):
    energie = 0.5 * liste_masse[i] * liste_geschwindigkeit[i]**2 / 1000 # Kj ausrechnen
    energie = round(energie,1) # Rundung
    liste_energie.append(energie)
    
df3['Energie [kJ]'] = liste_energie

In [ ]:
#df3.profile_report()

## 6. Statistik von df3

In [ ]:
df3.describe()
df3.describe(include=['object'])
df3.describe(include='all')

### Statistik der Masse

In [ ]:
varianz = df3["Masse [kg]"].var()
standardabweichung = df3["Masse [kg]"].std()
schiefemasse = df3["Masse [kg]"].skew()
kurtosis = df3["Masse [kg]"].kurtosis()

In [ ]:
print(("Varianz {0}, Standardabweichung {1}, Schiefemasse {2}, Kurtosis {3}").format(varianz, standardabweichung, schiefemasse, kurtosis))

## Histogramme von df3

In [ ]:
df3.head(5)

### Histogram Masse

In [2]:
bins = [x for x in range(0,3200,50)]

plt.hist(df3["Masse [kg]"])

plt.xticks(bins)

plt.xlabel("Masse [kg]")
plt.ylabel("Anzahl Steine")
plt.title("Histogramme Masse der Steine")

plt.show()

NameError: name 'plt' is not defined

### Histogram Geschwindigkeit

In [ ]:
bins = [x for x in range(0,3200,50)]

plt.hist(df3["Geschwindigkeit [m/s]"])

plt.xticks(bins)

plt.xlabel("Geschwindigkeit [m/s]")
plt.ylabel("Anzahl Geschwindigkeit")
plt.title("Histogramme Geschwindigkeit")

plt.show()

### Histogram Energie

In [ ]:
bins = [x for x in range(0,3200,50)]

plt.hist(df3["Energie [kJ]"])

plt.xticks(bins)

plt.xlabel("Energie [kJ]")
plt.ylabel("Anzahl Energie")
plt.title("Histogramme Energie")

plt.show()